# Sistema RAG para Consultas a Documentos PDF

**Estudiante:** Jeison David Jiménez Alvear

**Código:** T00054331

**Curso:** Procesamiento del lenguaje natural

**Institución:** Universidad Tecnológica de Bolívar

---

Este notebook implementa un sistema de Recuperación Aumentada con Generación (RAG, por sus siglas en inglés) para responder preguntas sobre el contenido de documentos PDF. El objetivo es combinar la recuperación de información semántica con modelos generativos, permitiendo generar respuestas más precisas, basadas en fragmentos relevantes del documento.

Este enfoque es especialmente útil cuando se necesita consultar documentos largos o técnicos de forma conversacional o automatizada.


## 🧭 Flujo General del Proceso

1. **Instalación de dependencias**
   - Se instalan las bibliotecas necesarias como `PyMuPDF`, `OpenAI`, `FAISS` y `scikit-learn`.

2. **Extracción de texto desde archivos PDF** (`extract_text_from_pdf`)
   - Se define una función que lee el contenido textual de un PDF utilizando `PyMuPDF`.

3. **Fragmentación del texto (chunking)** (`extract_and_chunk_text_from_pdf`)
   - El texto extraído se divide en fragmentos con solapamiento para preservar el contexto y mejorar la recuperación.

4. **Creación del índice vectorial** (`create_index`)
   - Se convierten los fragmentos en vectores usando embeddings y se indexan con FAISS, lo que permite realizar búsquedas semánticas eficientes.

5. **Búsqueda de información relevante** (`search_index`)
   - Ante una pregunta del usuario, se busca en el índice FAISS los fragmentos más relevantes relacionados con la consulta.

6. **Generación de respuestas** (`generate_answer`)
   - Se utiliza un modelo generativo (como GPT) para generar una respuesta basada en los fragmentos recuperados, proporcionando una salida contextualizada y útil.


In [1]:
!pip install openai pymupdf faiss-cpu scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 17.7 MB/s eta 0:00:00


In [2]:
"""
Sistema RAG (Retrieval-Augmented Generation) para consultas sobre documentos PDF.
Este script permite cargar documentos PDF, indexarlos y generar respuestas
enriquecidas utilizando un modelo de lenguaje.
"""

import os
import fitz  # PyMuPDF
import faiss
import numpy as np
from openai import OpenAI
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Extrae texto de un archivo PDF.

    Args:
        pdf_path (str): Ruta al archivo PDF

    Returns:
        str: Texto extraído del PDF
    """
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

In [4]:
def extract_and_chunk_text_from_pdf(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    Extrae texto de un PDF y lo divide en fragmentos más pequeños (chunks)
    con cierto solapamiento para mantener el contexto.

    Args:
        pdf_path (str): Ruta al archivo PDF
        chunk_size (int): Tamaño aproximado de cada fragmento en caracteres
        chunk_overlap (int): Cantidad de caracteres que se solapan entre fragmentos

    Returns:
        list: Lista de fragmentos de texto con metadatos
    """
    # Extraer texto completo
    doc = fitz.open(pdf_path)
    full_text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        full_text += page.get_text()

    # Dividir en chunks
    chunks = []
    doc_name = os.path.basename(pdf_path)

    # Podemos dividir por párrafos para mantener coherencia
    paragraphs = full_text.split('\n\n')
    current_chunk = ""

    for paragraph in paragraphs:
        paragraph = paragraph.strip()
        if not paragraph:
            continue

        # Si añadir este párrafo excede el tamaño del chunk, guardamos el chunk actual
        if len(current_chunk) + len(paragraph) > chunk_size and current_chunk:
            chunks.append({
                "text": current_chunk,
                "source": doc_name,
                "chunk_id": len(chunks)
            })
            # Mantener algo del texto anterior para preservar contexto
            current_chunk = current_chunk[-chunk_overlap:] if chunk_overlap > 0 else ""

        # Añadir el párrafo al chunk actual
        if current_chunk:
            current_chunk += "\n\n" + paragraph
        else:
            current_chunk = paragraph

    # Añadir el último chunk si contiene texto
    if current_chunk:
        chunks.append({
            "text": current_chunk,
            "source": doc_name,
            "chunk_id": len(chunks)
        })

    return chunks


In [5]:
def create_index(documents):
    """
    Crea un índice vectorial FAISS a partir de una lista de documentos.

    Args:
        documents (list): Lista de documentos de texto

    Returns:
        tuple: (índice FAISS, vectorizador TF-IDF)
    """
    vectorizer = TfidfVectorizer()
    doc_vectors = vectorizer.fit_transform(documents).toarray()

    # Crear índice FAISS
    dimension = doc_vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(doc_vectors)

    return index, vectorizer

In [6]:
def search_documents(query, index, vectorizer, documents, top_k=3):
    """
    Busca documentos relevantes para una consulta dada.

    Args:
        query (str): Consulta del usuario
        index (faiss.Index): Índice FAISS
        vectorizer (TfidfVectorizer): Vectorizador TF-IDF
        documents (list): Lista de documentos originales
        top_k (int): Número de resultados a devolver

    Returns:
        list: Lista de tuplas (documento, puntuación)
    """
    query_vector = vectorizer.transform([query]).toarray()

    # Asegurarse de que top_k no sea mayor que el número de documentos
    actual_top_k = min(top_k, len(documents))

    if actual_top_k == 0:
        print("ADVERTENCIA: No hay documentos disponibles para buscar.")
        return []

    distances, indices = index.search(query_vector, actual_top_k)

    # Filtrar índices inválidos
    valid_results = []
    for i, idx in enumerate(indices[0]):
        if 0 <= idx < len(documents):  # Verificar que el índice sea válido
            valid_results.append((documents[idx], distances[0][i]))

    return valid_results

In [7]:
def generate_augmented_response(query, index, vectorizer, documents, api_key):
    """
    Genera una respuesta aumentada utilizando RAG.

    Args:
        query (str): Consulta del usuario
        index (faiss.Index): Índice FAISS
        vectorizer (TfidfVectorizer): Vectorizador TF-IDF
        documents (list): Lista de documentos originales
        api_key (str): Clave API de OpenAI

    Returns:
        str: Respuesta generada
    """
    # Configurar cliente de OpenAI con la nueva API
    client = OpenAI(api_key=api_key)

    # Buscar documentos relevantes
    search_results = search_documents(query, index, vectorizer, documents)

    if not search_results:
        return "No se encontraron documentos relevantes para responder a tu consulta."

    # Preparar contexto con los documentos recuperados
    context_docs = [f"Documento {i+1}:\n{result[0][:1000]}"
                   for i, result in enumerate(search_results)]
    context = "\n\n".join(context_docs)

    # Crear un prompt estructurado para el modelo

    system_prompt = (
    "You are a research assistant who answers questions based on "
    "the provided information. Use only the information from the "
    "supplied documents. If the information is insufficient to "
    "answer, indicate it clearly."
    )

    user_prompt = (
    f"I have found these relevant document excerpts for your question:\n\n"
    f"{context}\n\n"
    f"Based solely on these documents, answer the following question: "
    f"{query}"
    )

    try:
        # Generar respuesta con la nueva API
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=300,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error al generar respuesta: {str(e)}"

In [8]:
def main():
    """Función principal del programa (versión fuera de Colab)."""
    print("=== Sistema RAG para consulta de documentos PDF ===")

    # Carpeta donde están los PDFs
    pdf_directory = "/content/files" #Colocar la ruta a la carpeta con los archivos .pdf

    if not os.path.exists(pdf_directory):
        print(f"La carpeta {pdf_directory} no existe. Crea la carpeta y coloca allí tus archivos PDF.")
        return

    # Listar los archivos PDF
    pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith(".pdf")]
    if not pdf_files:
        print(f"No se encontraron archivos PDF en la carpeta {pdf_directory}.")
        return

    # Extraer texto y fragmentar
    all_chunks = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_directory, pdf_file)
        print(f"Procesando {pdf_file}...")
        try:
            chunks = extract_and_chunk_text_from_pdf(pdf_path)
            all_chunks.extend(chunks)
            print(f"  - Fragmentos extraídos: {len(chunks)}")
        except Exception as e:
            print(f"  - Error al procesar {pdf_file}: {str(e)}")

    if not all_chunks:
        print("No se pudo extraer texto de ningún archivo PDF.")
        return

    print("\n=== Fragmentos generados ===")
    for i, chunk in enumerate(all_chunks[:3], 1):
        print(f"{i}. {chunk['source']} - {len(chunk['text'])} caracteres")
        preview = chunk['text'][:200].replace('\n', ' ') + "..."
        print(f"   Vista previa: {preview}")

    chunk_texts = [chunk["text"] for chunk in all_chunks]

    print("\nCreando índice de búsqueda...")
    index, vectorizer = create_index(chunk_texts)
    print(f"Índice creado con {len(chunk_texts)} fragmentos.")

    # Solicitar clave de API (opcional)
    api_key = 'Ingresa tu api key aqui'

    print("\n=== Sistema listo para consultas ===")
    print("Puedes hacer preguntas sobre los documentos cargados.")

    while True:
        query = input("\nIngresa tu consulta (o 'salir' para terminar): ")
        if query.lower() in ['salir', 'exit', 'quit']:
            break

        if not query.strip():
            print("Por favor, ingresa una consulta válida.")
            continue

        print("\nBuscando información relevante...")
        try:
            if api_key:
                response = generate_augmented_response(query, index, vectorizer, chunk_texts, api_key)
                print("\nRespuesta:")
                print(response)
            else:
                results = search_documents(query, index, vectorizer, chunk_texts)
                print("\nFragmentos relevantes encontrados:")
                for i, (doc, score) in enumerate(results, 1):
                    print(f"\n--- Fragmento {i} (Relevancia: {1 - score:.4f}) ---")
                    print(doc[:300] + "...")
        except Exception as e:
            print(f"Error al procesar la consulta: {str(e)}")

    print("\n¡Gracias por usar el sistema RAG!")


if __name__ == "__main__":
    main()


# Questions examples
 # 1. What does the system proposed by the SINAI group for gambling detection consist of?
 # 2. What were the performance metrics obtained by the SINAI group with its gambling detection system?

=== Sistema RAG para consulta de documentos PDF ===
La carpeta /content/files no existe. Crea la carpeta y coloca allí tus archivos PDF.
